# Silver Layer – Customer Transactions

## Purpose
This notebook cleans and transforms raw Online Retail data from the Bronze layer
into structured, customer-level order transactions suitable for analytics and
machine learning.

## Input
- Bronze table:
  workspace.repeat_purchase.bronze_online_retail

## Key Transformations
- Convert invoice date from string to timestamp
- Remove guest customers (null CustomerID)
- Remove returns and cancellations (Quantity <= 0)
- Aggregate product-level rows to order-level transactions
- Calculate total order quantity and order value

## Output
- Silver table:
  workspace.repeat_purchase.silver_customer_transactions

## Notes
- Each row represents one customer order
- Repeat purchase labeling will be handled in the Gold layer


In [0]:
# Read raw Bronze data

df_bronze = spark.table("workspace.repeat_purchase.bronze_online_retail")


In [0]:
# Convert InvoiceDate

from pyspark.sql.functions import col, to_timestamp

df_clean_dates = df_bronze.withColumn(
    "invoice_ts",
    to_timestamp(col("InvoiceDate"), "dd-MM-yyyy HH:mm")
)

In [0]:
# Remove null customers

df_with_customers = df_clean_dates.filter(
    col("CustomerID").isNotNull()
)


In [0]:
# Remove returns

df_positive_qty = df_with_customers.filter(
    col("Quantity") > 0
)


In [0]:
# Aggregate to order level

from pyspark.sql.functions import sum as spark_sum

df_orders = (
    df_positive_qty
    .groupBy("CustomerID", "InvoiceNo", "invoice_ts")
    .agg(
        spark_sum(col("Quantity")).alias("total_quantity"),
        spark_sum(col("Quantity") * col("UnitPrice")).alias("order_value")
    )
)


In [0]:
# Write Silver table

(
    df_orders.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("workspace.repeat_purchase.silver_customer_transactions")
)


In [0]:
# Final verification (display)

display(
    spark.sql("""
    SELECT COUNT(*) AS order_count
    FROM workspace.repeat_purchase.silver_customer_transactions
    """)
)


order_count
18566
